In [1]:
import pandas as pd
import shutil


# NY_AA_05_24_24

We were asked to calculate the ten largest asian ethnic groups in each New York County

In [15]:
# Load data, filter to counties
h = pd.read_csv("./raw-from-source/2020-ddhc-a/ddhca_t01001.csv")
counties = h[h["REGION_TYPE"]=="COUNTY"]

# Create a dictionary matching id to ethnic group
vals = pd.read_excel("./raw-from-source/2020-census-hispanic-origin-and-race-iterations-list.xlsx", header = None)
vals_dict = dict(zip(vals[0].astype(str),vals[1]))

In [6]:
# Group by county and group, reset index
new_file = counties.groupby(["REGION_ID","ITERID"])[["COUNT"]].sum()
new_file.reset_index(inplace = True, drop = False)

# Filter to asian ethnicities, clean id
asian = new_file[(new_file["ITERID"]>=3821)&(new_file["ITERID"]<=3876)]
asian["REGION_ID"] = asian["REGION_ID"].astype(str).str.zfill(5)

# Filter to NY counties
asian = asian[asian["REGION_ID"].str[0:2]=="36"]
asian["ITERID"] = asian["ITERID"].astype(str)

# Add in group names
asian["Name"] = asian["ITERID"].map(vals_dict)

# Filter out combination groups
asian = asian[~asian["Name"].str.contains("\*")]

# Get the top 10 largest by county
grouped = asian.groupby('REGION_ID').apply(lambda x : x.sort_values(by = 'COUNT', ascending = False).head(10).reset_index(drop = True))

/var/folders/1t/0q4w6hm92mg_zxd84dfxmq3m0000gn/T/ipykernel_52033/899628847.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asian["REGION_ID"] = asian["REGION_ID"].astype(str).str.zfill(5)
/var/folders/1t/0q4w6hm92mg_zxd84dfxmq3m0000gn/T/ipykernel_52033/899628847.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asian["ITERID"] = asian["ITERID"].astype(str)
/var/folders/1t/0q4w6hm92mg_zxd84dfxmq3m0000gn/T/ipykernel_52033/899628847.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

In [7]:
# Clean up indices and add rank
grouped = grouped.droplevel('REGION_ID')
grouped.reset_index(inplace = True, drop = False)
grouped.rename(columns = {"index":"rank"}, inplace = True)
grouped["rank"] +=1
grouped.reset_index(inplace = True, drop = True)

In [16]:
# Add a county name
county_names = pd.read_csv("/Users/peterhorton/Documents/RDH/raw_data/census/PL_COUNTYFP_NAMES.csv")
ny = county_names[county_names["STUSAB"]=="NY"]
ny["UNQ"] = "36" + ny["COUNTYFP"].astype(str).str.zfill(3)
ny_dict = dict(zip(ny["UNQ"], ny["NAME"]))
grouped["County"] = grouped["REGION_ID"].map(ny_dict)

/var/folders/1t/0q4w6hm92mg_zxd84dfxmq3m0000gn/T/ipykernel_52033/1619836722.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ny["UNQ"] = "36" + ny["COUNTYFP"].astype(str).str.zfill(3)


In [9]:
# Filter columns, export to csv
grouped = grouped[["County","REGION_ID","Name","ITERID","COUNT","rank"]]
grouped.to_csv("./ny_cnty_2020_asian_ethnicities/ny_cnty_2020_asian_ethnicities.csv", index = False)

# Make archive
shutil.make_archive("./ny_cnty_2020_asian_ethnicities","zip","./ny_cnty_2020_asian_ethnicities")